In [1]:
# scraping: https://www.northeastern.edu/graduate/?s=data+analytics&paged=1

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools
import re

In [3]:
program=[]
base_url='https://www.northeastern.edu/graduate/?s=master&paged={}'

for n in range(1,30):
    scrape_url=base_url.format(n)
    res=requests.get(scrape_url)

    soup=bs(res.text, 'lxml')
    courses=soup.select('.program-title')
    duration=soup.select('.col-3')
    location=soup.select('.col-1')
    
    for loc, year, course in zip(location, duration, courses): 
        course_title=course.select('a')[0] 
        course_duration=list(year.select('nobr'))
        course_location=list(loc.select('h5'))
        
        if len(course_location)==1:
            program.append([course_title.text, course_duration[0].text,course_location[0].text ,course_title['href']])
    
programdf = pd.DataFrame(program,columns = ["Program Name","Duration",'loc', "Program Link"])

programdf=programdf.drop('loc', axis=1)

programdf.drop_duplicates(subset="Program Name", keep="first", inplace=True)
programdf

,Program Name,Duration,Program Link
0,Cybersecurity,2-3 Years,https://www.northeastern.edu/graduate/program/...
1,Bioengineering,2-2.5 years,https://www.northeastern.edu/graduate/program/...
2,Health Informatics,2 Years,https://www.northeastern.edu/graduate/program/...
3,Computer Science,2-3 Years,https://www.northeastern.edu/graduate/program/...
4,Electrical and Computer Engineering,18-24 Months,https://www.northeastern.edu/graduate/program/...
...,...,...,...
117,Organizational Leadership/1-Year Experiential,1-2 years,https://www.northeastern.edu/graduate/program/...
119,Robotics,2 Years,https://www.northeastern.edu/graduate/program/...
121,Insurance Analytics and Management,2.5-3 Years,https://www.northeastern.edu/graduate/program/...
122,Applied Machine Intelligence,2.5-3 Years,https://www.northeastern.edu/graduate/program/...


In [4]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [5]:

for i,row in programdf.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.neu_program (program_name,duration,program_url) values(%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted


In [6]:
# scraping https://catalog.northeastern.edu/course-descriptions/****/ for DAMG/INFO/CSYE

In [7]:
course_catalog=[]
url_damg='https://catalog.northeastern.edu/course-descriptions/damg/'

result_damg=requests.get(url_damg)
soup_catalog_damg=bs(result_damg.text,'lxml')
catalog_damg=soup_catalog_damg.select(".courseblock")

for course in catalog_damg:
    
    course_id=course.select('.courseblocktitle')[0]
    course_description=course.select('.cb_desc')[0]
    
    c_id=re.search(r'\d\d\d\d', course_id.text)
    damg_id='DAMG '+ str(c_id.group())
    damg_name=course_id.text[12:]
        
    course_catalog.append(['Data Analytics Engineering',damg_id, damg_name,course_description.text])
    
    
url_info='https://catalog.northeastern.edu/course-descriptions/info/'

result_info=requests.get(url_info)
soup_catalog_info=bs(result_info.text,'lxml')
catalog_info=soup_catalog_info.select(".courseblock")

for course in catalog_info:
    
    course_id=course.select('.courseblocktitle')[0]
    course_description=course.select('.cb_desc')[0]
    
    c_id=re.search(r'\d\d\d\d', course_id.text)
    info_id='INFO '+ str(c_id.group())
    info_name=course_id.text[12:]
        
    course_catalog.append(['Information Systems',info_id, info_name,course_description.text])

url_csye='https://catalog.northeastern.edu/course-descriptions/csye/'

course_catalog_csye=[]

result_csye=requests.get(url_csye)
soup_catalog_csye=bs(result_csye.text,'lxml')
catalog_csye=soup_catalog_csye.select(".courseblock")

for course in catalog_csye:
    
    course_id=course.select('.courseblocktitle')[0]
    course_description=course.select('.cb_desc')[0]
    
    c_id=re.search(r'\d\d\d\d', course_id.text)
    csye_id='CSYE '+ str(c_id.group())
    csye_name=course_id.text[12:]
        
    course_catalog.append(['Software Engineering Systems', csye_id, csye_name,course_description.text])

In [8]:
catalogdf = pd.DataFrame(course_catalog,columns = ["Program_name","Course ID","Course_Name", "Description"])
catalogdf.drop_duplicates(subset="Course ID", keep="first", inplace=True)
catalogdf

,Program_name,Course ID,Course_Name,Description
0,Data Analytics Engineering,DAMG 6105,Data Science Engineering with Python. (4 Hours),Studies the Python programming language with d...
1,Data Analytics Engineering,DAMG 6210,Data Management and Database Design. (4 Hours),Studies design of information systems from a d...
2,Data Analytics Engineering,DAMG 7105,Intelligent Data Modeling and Presentation for...,Studies every step in a data analysis pipeline...
3,Data Analytics Engineering,DAMG 7245,Big-Data Systems and Intelligence Analytics. ...,Offers students an opportunity to learn a hand...
4,Data Analytics Engineering,DAMG 7250,Big Data Architecture and Governance. (4 Hours),Focuses on creating and managing a data-driven...
...,...,...,...,...
75,Software Engineering Systems,CSYE 7550,Distributed Intelligent Agents in the Metavers...,Introduces the field of agent-based computing ...
76,Software Engineering Systems,CSYE 7962,Elective. (1-4 Hours),Offers elective credit for courses taken at ot...
77,Software Engineering Systems,CSYE 7978,Independent Study. (1-4 Hours),Offers theoretical or experimental work under ...
78,Software Engineering Systems,CSYE 7990,Thesis. (1-8 Hours),Offers analytical and/or experimental work con...


In [9]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [10]:

for i,row in catalogdf.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.neu_course_catalog (program_name,course_id,course_name,course_description) values(%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted


In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('Resource_material.csv',
                usecols = ['Please select the course id for above course.',
       'Please provide website url for the above software download',
       'Which Professional Certifications are available to master skills for this course?',
       'Which online platform is available  to practice/study the course?',
       'Course Rating ', 'ResourceType'])

In [13]:
df.head()

,Please select the course id for above course.,Please provide website url for the above software download,Which Professional Certifications are available to master skills for this course?,Which online platform is available to practice/study the course?,Course Rating,ResourceType
0,INFO 6106,https://neural-designer.en.softonic.com/,Azure AI Engineer Associate: Certificate from...,https://playground.tensorflow.org/#activation=...,4,Free
1,DAMG 6105,https://www.python.org/downloads/,Certified Professional in Python Programming 1,https://www.online-python.com/,3,Free
2,CSYE 6225,https://azure.microsoft.com/en-us/downloads/,AZ-900 Microsoft Azure Fundamentals,https://www.cloudzero.com/platform,5,Paid
3,INFO 5002,https://www.python.org/downloads/,PCEP: Certified Entry-Level Python Programmer,https://www.online-python.com/,3,Free
4,INFO 6205,https://visualstudio.microsoft.com/,https://www.udemy.com/course/c-sharp-visual-st...,Udemy,4,Free


In [14]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [15]:

for i,row in df.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.resource_material1(course_id,software_download_url,professional_certificate,online_platform,course_rating,resource_type) values (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted


In [16]:
# Reading faculty details from Google response sheet

In [17]:
faculty = pd.read_csv('Faculty.csv',
                usecols = ['Which Program did you take in Northeastern?',
                          'Which course did you completed?',
                          'Under Which professor did you choose your course?',
                          'Rate the professor'])

In [18]:
faculty.head ()

,Which Program did you take in Northeastern?,Which course did you completed?,Under Which professor did you choose your course?,Rate the professor
0,Information Systems,INFO 5100,Prof Khaled Bugrara,4
1,Data Analytics Engineering,DAMG 6105,Prof Nick Brown,4
2,Data Analytics Engineering,DAMG 7350,Nick Beauchamp,4
3,Information Systems,INFO 6105,Nik Bear Brown,5
4,Software Engineering Systems,CSYE 6305,Prof. Hossein Noorian,3


In [19]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [20]:

for i,row in faculty.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.NEU_Course_Faculty (program_name,course_id,faculty_name,faculty_rating) values(%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted


In [21]:
# Scrape data for finding core_course_requirements

In [22]:
# importing the libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd


urls=['https://catalog.northeastern.edu/graduate/engineering/multidisciplinary/information-systems-msis/#programrequirementstext','https://catalog.northeastern.edu/graduate/engineering/multidisciplinary/software-engineering-systems-ms/#programrequirementstext','https://catalog.northeastern.edu/graduate/engineering/mechanical-industrial/data-analytics-engineering-ms/#programrequirementstext']

program_name = {'https://catalog.northeastern.edu/graduate/engineering/multidisciplinary/information-systems-msis/#programrequirementstext': 'Information Systems', 'https://catalog.northeastern.edu/graduate/engineering/multidisciplinary/software-engineering-systems-ms/#programrequirementstext': 'Software Engineering Systems', 'https://catalog.northeastern.edu/graduate/engineering/mechanical-industrial/data-analytics-engineering-ms/#programrequirementstext': 'Data Analytics Engineering'}

tableValues=[]

for url in urls:
    core_course=[]
    result=requests.get(url)
    soup=bs(result.text,'html.parser')
    core_course_catalog=soup.select(".sc_courselist")


    myTable = soup.find('table',{'class':"sc_courselist"})

    row_headers = []
    for x in myTable.find_all('tr'):
        for y in x.find_all('th'):
            row_headers.append(y.text)


    for x in myTable.find_all('tr')[1:]:
        td_tags = x.find_all('td')
        td_val = [program_name[url]]+[y.text for y in td_tags]
        tableValues.append(td_val)
    
df=pd.DataFrame(tableValues,columns=['Program Name','Course ID','Course Name','Credit Hours'])

df=df.drop('Credit Hours', axis=1)
df

,Program Name,Course ID,Course Name
0,Information Systems,INFO 5100and INFO 5101,Application Engineering and Developmentand Lab...
1,Software Engineering Systems,CSYE 6200,Concepts of Object-Oriented Design
2,Software Engineering Systems,INFO 6205,Program Structure and Algorithms
3,Data Analytics Engineering,IE 5374,Special Topics in Industrial Engineering
4,Data Analytics Engineering,IE 6600,Computation and Visualization for Analytics
5,Data Analytics Engineering,IE 6700,Data Management for Analytics
6,Data Analytics Engineering,or DAMG 6210,Data Management and Database Design
7,Data Analytics Engineering,IE 7275,Data Mining in Engineering
8,Data Analytics Engineering,IE 7280,Statistical Methods in Engineering
9,Data Analytics Engineering,OR 6205,Deterministic Operations Research


In [23]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [24]:

for i,row in df.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.course_core_requirement (program_name,course_id,course_name) values (%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted


In [25]:
#import pandas as pd
#df = pd.DataFrame(pd.read_json("C:\\Users\\utkar\\OneDrive\\Desktop\\SEM 1\\Java\\Java_Assignments\\linkedinbot\\linkedinbot\\spiders\\data\\linkedinSpider_2022-12-01T04-41-04.json"))
#df

In [26]:
#create dataframe for jobs

In [27]:
df_SES = pd.DataFrame(pd.read_json("C:\\Users\\utkar\\OneDrive\\Desktop\\SEM 1\\Java\\Java_Assignments\\linkedinbot\\linkedinbot\\spiders\\data\\linkedinSpider_2022-12-03T22-03-06.json"))
df_IS = pd.DataFrame(pd.read_json("C:\\Users\\utkar\\OneDrive\\Desktop\\SEM 1\\Java\\Java_Assignments\\linkedinbot\\linkedinbot\\spiders\\data\\linkedinSpider_2022-12-03T22-07-03.json"))
df_DAE = pd.DataFrame(pd.read_json("C:\\Users\\utkar\\OneDrive\\Desktop\\SEM 1\\Java\\Java_Assignments\\linkedinbot\\linkedinbot\\spiders\\data\\linkedinSpider_2022-12-03T22-10-03.json"))
df=pd.concat([df_SES, df_IS, df_DAE])
df

,programName,title,jobId,companyName,location,datePosted,link,description,seniorityLevel,employmentType,jobFunction,industry
0,Data Analytics Engineering,"Associate, Content Analytics & Operations",3356615002,Netflix,"Los Angeles, CA",2022-11-16,https://www.linkedin.com/jobs/view/associate-c...,Netflix is the world's leading streaming enter...,Not Applicable,Full-time,Management and Manufacturing,"Movies, Videos, and Sound, Technology, Informa..."
1,Data Analytics Engineering,"Principal Analyst, Data Science & Analytics",3311157304,Zillow,United States,2022-11-23,https://www.linkedin.com/jobs/view/principal-a...,About The Team The Zillow Analytics organizati...,Mid-Senior level,Full-time,Engineering and Science,Software Development
2,Data Analytics Engineering,"VP, Data Analytics",3339080809,TechInsights,"Denver, CO",2022-11-01,https://www.linkedin.com/jobs/view/vp-data-ana...,OUR STORY TechInsights is the world’s most tru...,Director,Full-time,Business Development and Sales,Internet Publishing
3,Data Analytics Engineering,Vice President – Data Science & Analytics,3219049068,H&R Block,United States,2022-11-28,https://www.linkedin.com/jobs/view/vice-presid...,What You'll Do... H&R Block is seeking a Vice ...,Executive,Full-time,Research,Financial Services and Retail
4,Data Analytics Engineering,"VP, Data Analytics",3343511133,Propio Language Services,"Overland Park, KS",2022-11-08,https://www.linkedin.com/jobs/view/vp-data-ana...,At Propio Language Services we are closing t...,Executive,Full-time,Business Development and Sales,Translation and Localization
...,...,...,...,...,...,...,...,...,...,...,...,...
45,Software Engineering Systems,"Vice President, System Operations and Infrastr...",3379639408,Raintree Systems,United States,2022-11-29,https://www.linkedin.com/jobs/view/vice-presid...,As VP of Systems Operations & Infrastructure f...,Director,Full-time,Information Technology,Internet Publishing
46,Software Engineering Systems,Information Technology Engineer,3385840807,Outsource IT Solutions,"Naperville, IL",None,https://www.linkedin.com/jobs/view/information...,We are looking for a dedicated and dynamic ind...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
47,Software Engineering Systems,Software Dev Engineering Intern,3385766806,Yahoo,United States,None,https://www.linkedin.com/jobs/view/software-de...,It takes powerful technology to connect our br...,Internship,Full-time,Engineering and Information Technology,Advertising Services
48,Software Engineering Systems,Systems Developer,3385236440,Ralco,"Marshall, MN",2022-11-08,https://www.linkedin.com/jobs/view/systems-dev...,Company Overview Ralco is a science and techno...,Mid-Senior level,Full-time,Engineering and Information Technology,Internet Publishing


In [28]:
import mysql.connector
neucourse = mysql.connector.connect(host = 'localhost', user = 'root',passwd = 'Sundaram@123', database = 'dmdd_proj_neu_course')
mycursor = neucourse.cursor()

In [29]:
for i,row in df.iterrows():
    sql = "INSERT INTO dmdd_proj_neu_course.jobs_info (program_name,title,job_id,company_name,location,date_posted,link,description,seniority_level,employement_type,job_function,industry) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    neucourse.commit()
print("Records inserted")

Records inserted
